In [8]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
HTML("""
<style>
.container { width:100% !important; }
</style>
""")

In [10]:
api_key = 'your api key'

In [11]:
def _get_data(symbols,time_from,time_to,api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

def _get_label_sentiment(x):
    if x <= -0.35:
        return 'Bearish','Bearish'
    elif -0.35 < x <= -0.15:
        return 'Somewhat-Bearish','Bearish'
    elif -0.15 < x < 0.15:
        return 'Neutral','Neutral'
    elif 0.15 <= x < 0.35:
        return 'Somewhat_Bullish','Bullish'
    else:  # x >= 0.35
        return 'Bullish','Bullish'

In [21]:
def get_dataset(time_from="20241007T1200",
                time_to='',
                MAX_API_CALLS_PER_DAY = 1, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 5 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%5==0:
            time.sleep(60)

        data=_get_data('TSLA',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract TSLA specific relevance (we didn't use it in video)
    df['ticker_relevance_TSLA']=df['ticker_sentiment'].apply(lambda l:[el for el in l if el['ticker']=='TSLA'][0]['relevance_score']).astype(float)
    # Extract TSLA specific sentiment
    df['ticker_sentiment_TSLA']=df['ticker_sentiment'].apply(lambda l:[el for el in l if el['ticker']=='TSLA'][0]['ticker_sentiment_score']).astype(float)
    # Only take tickers with TSLA in headline
    df=df[df.title.str.contains('tsla|tesla',case=False)]
    # Extract # of tickers
    df['num_tickers']=df.ticker_sentiment.apply(lambda l:len(l))
    # Only take when # of tickers = 1
    df = df[df.num_tickers==1]
    # Applying the function and creating two new columns
    df[['detailed_original_label','label']] = df.apply(lambda row: _get_label_sentiment(row['ticker_sentiment_TSLA']), axis=1, result_type='expand')
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    df.set_index('time_published',inplace=True)
    # Sort by time published
    df.sort_index(inplace=True)
    return df

In [22]:
df = get_dataset(time_to='')
df.to_csv('tsla_sentiment.csv')

In [23]:
df

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
time_published,,,,,,,,,,,,,,,,,
20241024T140300,ETFs to Tap Tesla's Solid Q3 Earnings,https://www.zacks.com/stock/news/2356847/etfs-...,[Sweta Killa],Tesla reports its biggest quarterly profit in ...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.284626,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.6764...",0.676457,0.431176,1,Bullish,Bullish
20241024T150857,Why Tesla Stock Is Soaring Today,https://www.fool.com/investing/2024/10/24/why-...,[Howard Smith],Tesla stock shot higher today after its third-...,https://g.foolcdn.com/editorial/images/795190/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.6...",0.309466,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.8779...",0.877990,0.510945,1,Bullish,Bullish
20241024T155500,3 Reasons to Buy Tesla Stock Like There's No T...,https://www.fool.com/investing/2024/10/24/3-re...,[Rick Munarriz],"Investors are liking Tesla's latest report, bu...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.226688,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.8088...",0.808852,0.332660,1,Somewhat_Bullish,Bullish
20241025T073809,Elon Musk Sees $33.5B Wealth Surge In A Day As...,https://www.benzinga.com/markets/equities/24/1...,[Pooja Rajkumari],Elon Musk experienced a dramatic increase in h...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.449339,Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.4950...",0.495015,0.445558,1,Bullish,Bullish
20241025T094250,Tesla Shares Slip In Pre-Market A Day After Ma...,https://www.benzinga.com/markets/equities/24/1...,[Pooja Rajkumari],Shares of Tesla Inc. TSLA experienced a declin...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.352444,Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.2712...",0.271274,0.124063,1,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20241115T160148,Dawn Project testing of Tesla Full Self-Drivin...,https://www.benzinga.com/pressreleases/24/11/g...,[Globe Newswire],Tesla fan confirms safety advocacy group's tes...,https://ml.globenewswire.com/Resource/Download...,Benzinga,General,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.037264,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8991...",0.899125,0.047364,1,Neutral,Neutral
20241116T133000,"Losing the $7,500 Electric Vehicle Credit Coul...",https://www.fool.com/investing/2024/11/16/how-...,[Travis Hoium],President-elect Trump wants to eliminate the $...,https://g.foolcdn.com/editorial/images/798036/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.294126,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.9591...",0.959190,0.598081,1,Bullish,Bullish
20241116T233000,Is Tesla Still a Millionaire-Maker Stock?,https://www.fool.com/investing/2024/11/16/is-t...,[Will Ebiefung],Shares in this electric vehicle leader have so...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.4...",0.334518,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5665...",0.566543,0.452913,1,Bullish,Bullish


In [24]:
df.shape

(76, 17)